# To make a analysis of finding the ultimate dating places in Kolkata

In [1]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


In [2]:
from IPython.display import Image 
from IPython.core.display import HTML 
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [1]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


## Finding the best place to date in city -- Kolkata

In [4]:
address = 'Kolkata'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/usr/lib/python3.7/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


22.5677459 88.3476023


## Now we need 5 factors to our hypothesis on the basis of finding the best date
  * Icecream --> ic_dataframe
  * Chinese  --> ch_dataframe
  * Italian  -->  it_dataframe
  * Park     --> pk_dataframe

## Finding places for ice cream

In [5]:
search_query = 'Ice cream'
radius = 5000
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
#url
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

# copying it to stationary shop dataframe

ic_dataframe = dataframe_filtered.copy()
ic_dataframe.head()

Ice cream .... OK!


,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Naturals Ice Cream,Ice Cream Shop,77/1A Park Street,IN,Kolkata,India,2077,"[77/1A Park Street, Kolkata 700016, West Benga...","[{'label': 'display', 'lat': 22.550341, 'lng':...",22.550341,88.354897,700016,West Bengal,5a251d6151950e28670124d5
1,Metro Ice Cream,Ice Cream Shop,NaN,IN,NaN,India,1646,[India],"[{'label': 'display', 'lat': 22.553568, 'lng':...",22.553568,88.352151,NaN,NaN,563a2d16cd103f210627133e
2,Thanco's Natural Ice Cream,Ice Cream Shop,"18A, Ram Mohan Dutta Rd",IN,Kolkata,India,3742,"[18A, Ram Mohan Dutta Rd, Kolkata 700020, West...","[{'label': 'display', 'lat': 22.53427080110658...",22.534271,88.351021,700020,West Bengal,59d6377bb23dfa72cc1059a9
3,The Cream & Fudge Factory,Ice Cream Shop,"3-A, Humayun Pl",IN,Kolkata,India,907,"[3-A, Humayun Pl, Kolkata 700087, West Bengal,...","[{'label': 'display', 'lat': 22.56065779513747...",22.560658,88.351954,700087,West Bengal,4efd6fb0d3e364d5fa15d32d
4,Fire N Ice,Italian Restaurant,"41, Jawaharlal Nehru Rd",IN,Kolkata,India,2050,"[41, Jawaharlal Nehru Rd, Kolkata 700071, West...","[{'label': 'display', 'lat': 22.54940945481822...",22.549409,88.349471,700071,West Bengal,4bfe9972b68d0f477e6dea57


In [6]:
# create map of Toronto using latitude and longitude values
map_kol_ic = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, cat, postcode in zip(ic_dataframe['lat'], ic_dataframe['lng'],  ic_dataframe['categories'],  ic_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#ef00eb',
        fill=True,
        fill_color='#f4b804',
        fill_opacity=0.7).add_to(map_kol_ic)  
    
map_kol_ic

## Finding places for Italian Food

In [7]:
search_query = 'Italian food'
radius = 5000
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
#url
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

# copying it to stationary shop dataframe

it_dataframe = dataframe_filtered.copy()
it_dataframe.head()

Italian food .... OK!


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Metro Food Plaza,Indian Restaurant,Jawaharlal Nehru Rd,IN,Kolkata,India,NaN,674,"[Jawaharlal Nehru Rd, Kolkata 700013, West Ben...","[{'label': 'display', 'lat': 22.56257989490831...",22.562580,88.351032,700013,West Bengal,5a6abef60a464d531adbbb27
1,Food Court,Food Court,Avani Riverside Mall,IN,Hāora,India,NaN,2507,"[Avani Riverside Mall, Hāora 711102, West Beng...","[{'label': 'display', 'lat': 22.56311490760389...",22.563115,88.323727,711102,West Bengal,4fd20db1e4b07915217167a0
2,Food Station,Fast Food Restaurant,"88, College St",IN,Kolkata,India,NaN,1699,"[88, College St, Kolkata 700073, West Bengal, ...","[{'label': 'display', 'lat': 22.57392730078463...",22.573927,88.362716,700073,West Bengal,51949e5a498eddc3ac97eed2
3,Savitri Fast Food Center,Fast Food Restaurant,NaN,IN,Kolkata,India,NaN,1240,"[Kolkata 700016, West Bengal, India]","[{'label': 'display', 'lat': 22.558012, 'lng':...",22.558012,88.353485,700016,West Bengal,5a6c6c5d9b047348f8e253b1
4,Savitri Food Centre,Fast Food Restaurant,NaN,IN,NaN,India,NaN,1247,[India],"[{'label': 'display', 'lat': 22.55769917978459...",22.557699,88.352986,NaN,NaN,53f36275498efdaea76db434


In [8]:
# create map of Toronto using latitude and longitude values
map_kol_it = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, cat, postcode in zip(it_dataframe['lat'], it_dataframe['lng'],  it_dataframe['categories'],  it_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#D49B17',
        fill=True,
        fill_color='#D49B17',
        fill_opacity=0.7).add_to(map_kol_it)  
    
map_kol_it

## Finding places for Chinese Food

In [9]:
search_query = 'Chinese food'
radius = 5000
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
#url
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

# copying it to stationary shop dataframe

ch_dataframe = dataframe_filtered.copy()
ch_dataframe.head()

Chinese food .... OK!


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Chinese Breakfast Market,Farmers Market,Surya Sen St,IN,Kolkata,India,NaN,917,"[Surya Sen St, Kolkata 700012, West Bengal, In...","[{'label': 'display', 'lat': 22.57299245418824...",22.572992,88.354486,700012,West Bengal,510480d9e4b0e45ffca02f95
1,Chinese Breakfast,Chinese Restaurant,Tiretty Bazaar,IN,NaN,India,NaN,1106,"[Tiretty Bazaar, India]","[{'label': 'display', 'lat': 22.57247906488648...",22.572479,88.357070,NaN,NaN,4df6bdf7b61c40d585eab4f8
2,Chinese Quisine Restaurant,Chinese Restaurant,Lake Town,IN,Kolkata,India,NaN,3743,"[Lake Town, Kolkata 700089, West Bengal, India]","[{'label': 'display', 'lat': 22.60137172155218...",22.601372,88.347486,700089,West Bengal,5124f7dbe4b08956fb32b6dc
3,Chinese Express and Takeaway Centre,Chinese Restaurant,"7/3, Mandeville Gardens",IN,Kolkata,India,NaN,5144,"[7/3, Mandeville Gardens, Kolkata 700019, West...","[{'label': 'display', 'lat': 22.52489591197098...",22.524896,88.366352,700019,West Bengal,5029af82e4b0f39571bedead
4,Metro Food Plaza,Indian Restaurant,Jawaharlal Nehru Rd,IN,Kolkata,India,NaN,674,"[Jawaharlal Nehru Rd, Kolkata 700013, West Ben...","[{'label': 'display', 'lat': 22.56257989490831...",22.562580,88.351032,700013,West Bengal,5a6abef60a464d531adbbb27


In [10]:
# create map of Toronto using latitude and longitude values
map_kol_ch = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, cat, postcode in zip(ch_dataframe['lat'], ch_dataframe['lng'],  ch_dataframe['categories'],  ch_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='Orange',
        fill_opacity=0.7).add_to(map_kol_ch)  
    
map_kol_ch

##  To find the parks

In [11]:
search_query = 'Park'
radius = 5000
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
#url
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

# copying it to stationary shop dataframe

pk_dataframe = dataframe_filtered.copy()

# cleaning the values only according to park
some_values = "Park"
#toronto_table.loc[toronto_table['Borough'].isin(some_values)]
#toronto_table.loc[toronto_table['Borough'] == some_values]
pk_dataframe = pk_dataframe[pk_dataframe['categories'].str.contains(some_values)]
pk_dataframe

Park .... OK!


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
2,Allen Park (Gopinath Sahay Udyan),Park,Mother Teresa Sarani,IN,Kolkata,India,Park St.,2062,"[Mother Teresa Sarani (Park St.), Kolkata 7000...","[{'label': 'display', 'lat': 22.55027437402718...",22.550274,88.354282,700016,West Bengal,4eeda2de7ee5f45387e80277
6,Elliot Park,Park,Jawaharlal Nehru Rd,IN,Kolkata,India,NaN,2036,"[Jawaharlal Nehru Rd, Kolkata 700071, West Ben...","[{'label': 'display', 'lat': 22.54946152695032...",22.549462,88.348274,700071,West Bengal,4ee474020e61681b96423352
10,Curzon park,Park,NaN,IN,NaN,India,NaN,456,[India],"[{'label': 'display', 'lat': 22.56377429545729...",22.563774,88.348700,NaN,NaN,516a47a7e4b01ebd2e763349
11,Deshapriya Park,Park,"5/2, Deshapriya Pk East",IN,Kolkata,India,NaN,5523,"[5/2, Deshapriya Pk East, Kolkata 700029, West...","[{'label': 'display', 'lat': 22.51839455820257...",22.518395,88.353177,700029,West Bengal,4ce40d96dc85a1432f6b49d2
14,Millennium Park,Park,"Strand Rd, B B D Bagh",IN,Kolkata,India,NaN,615,"[Strand Rd, B B D Bagh, Kolkata, West Bengal, ...","[{'label': 'display', 'lat': 22.57248103815183...",22.572481,88.344517,NaN,West Bengal,4d6f6d650187236afe939bba
15,Subhash Udyan (Northern Park),Park,Justice Chandra Madhab Rd,IN,Kolkata,India,NaN,3633,"[Justice Chandra Madhab Rd, Kolkata 700020, We...","[{'label': 'display', 'lat': 22.53520376097730...",22.535204,88.350350,700020,West Bengal,4e4fa145aeb70f1284a28703
16,Eden Garden Park,Park,Eden Garden Rd,IN,Kolkata,India,NaN,805,"[Eden Garden Rd, Kolkata 700007, West Bengal, ...","[{'label': 'display', 'lat': 22.56424437203329...",22.564244,88.340743,700007,West Bengal,50091879582f92d8b703b7a7
18,B K Pal Park,Park,NaN,IN,NaN,India,NaN,3291,[India],"[{'label': 'display', 'lat': 22.59518704410715...",22.595187,88.359540,NaN,NaN,510b2875e4b0898fa14370d6
19,Charlie Chaplin Park,Park,S. S. Hogg Mkt,IN,Kolkata,India,NaN,858,"[S. S. Hogg Mkt, Kolkata 700087, West Bengal, ...","[{'label': 'display', 'lat': 22.56159903290866...",22.561599,88.352652,700087,West Bengal,5a432e92f96b2c69e2532f05
21,Md. Ali Park,Park,Chittaranjan Ave,IN,Kolkata,India,NaN,1731,"[Chittaranjan Ave, Kolkata 700073, West Bengal...","[{'label': 'display', 'lat': 22.57750002609193...",22.577500,88.360720,700073,West Bengal,4fe0aa02be77cf38d7a19002


In [12]:
# create map of Toronto using latitude and longitude values
map_kol_pk = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, cat, postcode in zip(pk_dataframe['lat'], pk_dataframe['lng'],  pk_dataframe['categories'],  pk_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Green',
        fill=True,
        fill_color='Green',
        fill_opacity=0.7).add_to(map_kol_pk)  
    
map_kol_pk

## To colase all the maps

In [13]:
# create map of Toronto using latitude and longitude values
map_kol_date = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
# ice cream
for lat, lng, cat, postcode in zip(ic_dataframe['lat'], ic_dataframe['lng'],  ic_dataframe['categories'],  ic_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#ef00eb',
        fill=True,
        fill_color='#f4b804',
        fill_opacity=0.7).add_to(map_kol_date)  

# itlian food
for lat, lng, cat, postcode in zip(it_dataframe['lat'], it_dataframe['lng'],  it_dataframe['categories'],  it_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#D49B17',
        fill=True,
        fill_color='#D49B17',
        fill_opacity=0.7).add_to(map_kol_date)  

# chinese food
# add markers to map

#icon_url = 'https://cdn1.iconfinder.com/data/icons/maps-locations-2/96/Geo2-Number-512.png'
#icon = folium.features.CustomIcon(icon_url,icon_size=(28, 30))  # Creating a custom Icon
for lat, lng, cat, postcode in zip(ch_dataframe['lat'], ch_dataframe['lng'],  ch_dataframe['categories'],  ch_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    #folium.Marker(location=[lat, lng],icon=icon).add_to(map_kol_date)  #adding it to the map
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='Orange',
        fill_opacity=0.7).add_to(map_kol_date) 
# parks
# add markers to map
for lat, lng, cat, postcode in zip(pk_dataframe['lat'], pk_dataframe['lng'],  pk_dataframe['categories'],  pk_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Green',
        fill=True,
        fill_color='Green',
        fill_opacity=0.7).add_to(map_kol_date)  
map_kol_date

## To find the best place to date in kolkata according to cluster

In [14]:
from folium.plugins import MarkerCluster


# create map of Toronto using latitude and longitude values
map_kol_date = folium.Map(location=[latitude, longitude], zoom_start=11)
marker_cluster = MarkerCluster().add_to(map_kol_date)
# add markers to map
# ice cream
for lat, lng, cat, postcode in zip(ic_dataframe['lat'], ic_dataframe['lng'],  ic_dataframe['categories'],  ic_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#ef00eb',
        fill=True,
        fill_color='#f4b804',
        fill_opacity=0.7).add_to(marker_cluster)  

# itlian food
for lat, lng, cat, postcode in zip(it_dataframe['lat'], it_dataframe['lng'],  it_dataframe['categories'],  it_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#D49B17',
        fill=True,
        fill_color='#D49B17',
        fill_opacity=0.7).add_to(marker_cluster)  

# chinese food
# add markers to map

#icon_url = 'https://cdn1.iconfinder.com/data/icons/maps-locations-2/96/Geo2-Number-512.png'
#icon = folium.features.CustomIcon(icon_url,icon_size=(28, 30))  # Creating a custom Icon
for lat, lng, cat, postcode in zip(ch_dataframe['lat'], ch_dataframe['lng'],  ch_dataframe['categories'],  ch_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    #folium.Marker(location=[lat, lng],icon=icon).add_to(map_kol_date)  #adding it to the map
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='Orange',
        fill_opacity=0.7).add_to(marker_cluster) 
# parks
# add markers to map
for lat, lng, cat, postcode in zip(pk_dataframe['lat'], pk_dataframe['lng'],  pk_dataframe['categories'],  pk_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Green',
        fill=True,
        fill_color='Green',
        fill_opacity=0.7).add_to(marker_cluster)  
map_kol_date